In [4]:
from sys import path as sys_path
from os import chdir, curdir
from os import path as os_path

from glob import iglob
from itertools import product

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

HOME = os_path.expanduser("~")
# chdir(f"{HOME}/src/topology-programming")
chdir(f"/home/mhall7/durairajanlab.matt/topology-programming/")

sys_path.insert(0, "src/")
sys_path.insert(0, "src/utilities")

# from onset.utilities import plotters as my_plotters

report_folder = "data/archive/crossfire-rerun-02-01-24/reports/"
ECMP_palette = [sns.color_palette("tab10")[1], sns.color_palette("tab10")[0]]
MCF_palette = sns.color_palette("tab10")[2:4]


def export_legend(legend, filename="legend"):
    fig = legend.figure
    fig.canvas.draw()
    bbox = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(filename + ".pdf", dpi="figure", bbox_inches=bbox)


# Parameter dictionary
# plt.rcParams

result_df = pd.DataFrame()
all_columns = set()
for data_file in iglob(report_folder + "*.csv"):
    
    df = pd.read_csv(data_file)
    if len(df) == 1: 
        if "mcf" in data_file:
            print(data_file)
        
            print(df[["Iteration", "Experiment"]])
        
    all_columns.update(df.columns)
    result_df = pd.concat([result_df, df], ignore_index=True)

networks = {"bellCanada": "Bell Canada", "sprint": "Sprint", "surfNet": "Surf Net", "ANS": "ANS", "CRL": "CRL"}
result_df["Network"] = result_df["Experiment"].apply(lambda x: x.split("_")[0] if isinstance(x, str) else x)
result_df["Network"] = result_df["Network"].apply(lambda x: networks[x])

result_df["Routing"] = result_df["Routing"].apply(lambda x: "Ripple*" if x == "MCF" else x)

result_df["Strategy"] = result_df.apply(
    lambda row: row["Routing"] if row["Iteration"].split("-")[-1] == "0" else f"{row['Routing']}+ONSET", axis=1
)
result_df["Attack Scale"] = result_df["Experiment"].apply(lambda x: "100E9" if "100E9" in x else "200E9")


In [31]:
save_leg_ecmp = True
save_leg_ripple = True
for network, scale, routing in product(result_df["Network"].unique(), result_df["Attack Scale"].unique(), result_df["Routing"].unique()):
    plt.clf()    
    
    print(network, scale, routing)
    print()
    if scale == "200E9" and network == "Sprint" and routing == "ECMP": 
        print(temp[["Network", "Strategy", "Routing", "Congestion"]].sort_values(by="Strategy"))
        # data#
        break

    temp = result_df[result_df["Network"] == network]

    
    temp = temp[temp["Attack Scale"] == scale]
    # print(temp[["Network", "Strategy", "Routing", "Congestion"]])
    data = temp[temp["Routing"] == routing]


    # print(temp[["Network", "Strategy", "Routing", "Congestion"]])
    baseline_df = data[data["Strategy"] == routing]
    onset_df = data[data["Strategy"] == routing + "+ONSET"]

    # print(data[["Network", "Strategy", "Routing", "Congestion"]])    



    continue
    if routing == "ECMP": 
        palette = ECMP_palette
        legend_key = ["ECMP+ONSET" , "ECMP"]
    if routing == "Ripple*":
        palette = MCF_palette
        legend_key = ["Ripple*+ONSET", "Ripple*" ]
        
    # baseline_df = np.array(data_1["Congestion"])
    # onset_df = np.array(data_2["Congestion"])

    # data_dict = {"ECMP": X_1, "ECMP+ONSET": X_2}

    # my_plotters.congestion_multi_cdf(
    #     data_dict, inLabel="Max. Link Utilization", fig_name=f"data/plots/crossfire/congestion-{network}-{routing}-{scale}"
    # )

    plt.rc("font", size=22)
    figurename = f"data/plots/crossfire/congestion-{network}-{routing}-{scale}".replace(" ", "_")
    # )
    ax2 = sns.ecdfplot(data=onset_df, x="Congestion", linewidth=3, color=palette[1], label=list(onset_df["Strategy"])[0])
    ax1 = sns.ecdfplot(data=baseline_df, x="Congestion", linestyle="--", linewidth=3, color=palette[0], label=list(baseline_df["Strategy"])[0])
    
    # Set up the initial legend.
    my_legend = plt.legend(
        # [routing, routing + "+ONSET"],
        bbox_to_anchor=(-0.2, 2),
        loc="upper left",
        borderaxespad=0,
        ncol=2,
        frameon=False,
    )
    # handles, labels = my_legend.get_legend_handles_labels()
    plt.tick_params(length=8)

    if save_leg_ecmp and routing == "ECMP": 
        export_legend(my_legend, figurename + "_legend")
        save_leg_ecmp = False
    if save_leg_ripple and routing == "Ripple*": 
        export_legend(my_legend, figurename + "_legend")
        save_leg_ripple = False
    
    plt.legend().remove()
    plt.axvline(1, linestyle="--", color="black", linewidth=2)
    plt.ylabel("CDF")
    plt.xlabel("Max. Congestion")
    plt.tight_layout()
    plt.savefig(figurename + ".pdf")
    print(f"saved: {figurename}.pdf")
    
    with open(figurename + "_stats.txt", "w") as fob:
        fob.write("\tLoss Events\ttotal")
        fob.write(f"ONSET\t{len(onset_df[onset_df['Loss'] > 0])}\t{len(onset_df)}")
        fob.write(f"ONSET\t{len(onset_df[onset_df['Loss'] > 0])}\t{len(onset_df)}")
    print(f"saved: {figurename}_stats.txt")
    
    plt.clf()
    

Bell Canada 200E9 Ripple*

Bell Canada 200E9 ECMP

Bell Canada 100E9 Ripple*

Bell Canada 100E9 ECMP

CRL 200E9 Ripple*

CRL 200E9 ECMP

CRL 100E9 Ripple*

CRL 100E9 ECMP

Surf Net 200E9 Ripple*

Surf Net 200E9 ECMP

Surf Net 100E9 Ripple*

Surf Net 100E9 ECMP

Sprint 200E9 Ripple*

Sprint 200E9 ECMP

     Network       Strategy  Routing  Congestion
253   Sprint           ECMP     ECMP    1.789153
937   Sprint           ECMP     ECMP    1.344040
869   Sprint           ECMP     ECMP    1.365455
135   Sprint           ECMP     ECMP    2.560854
144   Sprint           ECMP     ECMP    0.119089
185   Sprint           ECMP     ECMP    1.838417
210   Sprint           ECMP     ECMP    1.818468
746   Sprint           ECMP     ECMP    1.677059
259   Sprint           ECMP     ECMP    0.625591
436   Sprint           ECMP     ECMP    2.402037
810   Sprint           ECMP     ECMP    1.726514
370   Sprint           ECMP     ECMP    2.549397
145   Sprint     ECMP+ONSET     ECMP    0.119089
260   Sprin

<Figure size 640x480 with 0 Axes>

In [33]:
660*2 - len(result_df)

291

In [ ]:
data_2[["Network", "Strategy", "Attack Scale", "Congestion", "Loss", "Throughput"]]

,Network,Strategy,Attack Scale,Congestion,Loss,Throughput
1,Bell Canada,ECMP+ONSET,100E9,0.793238,0.0,1.0
11,Bell Canada,ECMP+ONSET,100E9,0.474032,0.0,1.0
36,Bell Canada,ECMP+ONSET,100E9,0.170498,0.0,1.0
114,Bell Canada,ECMP+ONSET,100E9,0.488298,0.0,1.0
202,Bell Canada,ECMP+ONSET,100E9,0.652963,0.0,1.0
204,Bell Canada,ECMP+ONSET,100E9,0.572302,0.0,1.0
223,Bell Canada,ECMP+ONSET,100E9,0.527005,0.0,1.0
277,Bell Canada,ECMP+ONSET,100E9,0.498216,0.0,1.0
310,Bell Canada,ECMP+ONSET,100E9,0.723430,0.0,1.0
349,Bell Canada,ECMP+ONSET,100E9,0.757374,0.0,1.0
